In [ ]:
!pip install pyspark


In [8]:
import os
os.environ['PYARROW_IGNORE_TIMEZONE'] = '1'
import pandas as pd
import numpy as np
import pyspark.pandas as ps
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col,min,max,mean
import time
from pyspark.sql.functions import col, when, count, sum
import matplotlib.pyplot as plt
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType, TimestampType
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, count, sum, min, max, mean,countDistinct, desc
from pyspark.sql.types import IntegerType, FloatType
from pyspark.sql.functions import year, month, dayofmonth, hour, datediff, lit
from pyspark.sql import SparkSession
from pyspark.sql.functions import unix_timestamp, max as spark_max, sum as spark_sum, count as spark_count, col, udf
from pyspark.sql.window import Window
from pyspark.sql.types import IntegerType
import datetime



In [2]:


schema = StructType([
    StructField("event_time", TimestampType(), True),
    StructField("event_type", StringType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("category_id", LongType(), True),
    StructField("category_code", StringType(), True),
    StructField("brand", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("user_id", IntegerType(), True),
    StructField("user_session", StringType(), True)
])


In [3]:


spark = SparkSession.builder \
    .appName("clustering") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "5g") \
    .config("spark.executor.cores", "4") \
    .config("spark.driver.maxResultSize", "2g") \
    .getOrCreate()


24/06/18 21:03:42 WARN Utils: Your hostname, ghani-IdeaPad-3-14ADA05 resolves to a loopback address: 127.0.1.1; using 10.188.25.103 instead (on interface wlp2s0)
24/06/18 21:03:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/06/18 21:03:42 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/06/18 21:03:43 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/06/18 21:03:43 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/06/18 21:03:43 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.


In [5]:


## Charger les fichiers CSV avec le schéma défini
octobre_df = spark.read.csv("/home/ghani/Bureau/Data/2019-Oct.csv", header=True, schema=schema)
novembre_df = spark.read.csv("/home/ghani/Bureau/Data/2019-Nov.csv", header=True, schema=schema)
decembre_df = spark.read.csv("/home/ghani/Bureau/Data/2019-Dec.csv", header=True, schema=schema)
janvier_df = spark.read.csv("/home/ghani/Bureau/Data/2020-Jan.csv", header=True, schema=schema)
fevrier_df = spark.read.csv("/home/ghani/Bureau/Data/2020-Feb.csv", header=True, schema=schema)
mars_df = spark.read.csv("/home/ghani/Bureau/Data/2020-Mar.csv", header=True, schema=schema)
avril_df = spark.read.csv("/home/ghani/Bureau/Data/2020-Apr.csv", header=True, schema=schema)




In [6]:
combined_df = (octobre_df
               .union(novembre_df)
               .union(decembre_df)
               .union(janvier_df)
               .union(fevrier_df)
               .union(mars_df)
               .union(avril_df))

# Prendre un échantillon de 10 % du DataFrame combiné
#combined_sample = combined_df.sample(withReplacement=False, fraction=0.1, seed=42)






In [ ]:
#combined_sample.show()
#combined_sample.cache()
#combined_sample.count()
#combined_df.cache()
from pyspark.sql.functions import countDistinct

# Calculer le nombre d'éléments distincts dans la colonne 'user_id'
distinct_user_count = combined_df.select(countDistinct('user_id')).collect()[0][0]

print(f"Nombre d'utilisateurs distincts : {distinct_user_count}")




In [ ]:

data = combined_df.withColumn("event_time", unix_timestamp(col("event_time"), "yyyy-MM-dd HH:mm:ss").cast("timestamp"))

# Filtrer les événements d'achat
data_purchase = data.filter(col("event_type") == "purchase")

# Dernière date dans le jeu de données
last_date = data.agg(spark_max("event_time")).collect()[0][0] + datetime.timedelta(days=1)

# Fonction UDF pour calculer la récence
recency_udf = udf(lambda x: (last_date - x).days, IntegerType())

# Calculer RFM
rfm = data_purchase.groupBy("user_id").agg(
    recency_udf(spark_max("event_time")).alias("Recency"),
    spark_count("user_session").alias("Frequency"),
    spark_sum("price").alias("Monetary")
)

# Afficher les premières lignes des données RFM
rfm.show(5)

In [4]:
rfm2=spark.read.parquet('/home/ghani/Bureau/Data/rfm.parquet')

In [5]:
rfm2.cache()
rfm2.count()


2064899

In [20]:
rfm2.show(20)

+---------+-------+---------+------------------+
|  user_id|Recency|Frequency|          Monetary|
+---------+-------+---------+------------------+
|513724405|    189|        3|            272.86|
|543838936|    136|        2|            810.81|
|534285038|    171|        2|             79.75|
|517864279|    213|        1|            196.68|
|541774796|    213|        1|             29.51|
|555485382|    213|        1|            957.53|
|539702146|    117|        3|2458.2400000000002|
|512386086|    134|      806|473119.38999999984|
|545616788|     70|       88|          33947.31|
|513509085|     75|      155|180843.02999999997|
|513161483|    175|        6|           6235.88|
|513812036|     80|        6|            307.19|
|515494405|      2|       50|47981.880000000005|
|539319122|     77|        6|           1289.26|
|550397911|     17|        2|            109.14|
|519311105|    213|        1|            111.79|
|513723185|     54|        7|           1281.94|
|536615687|    169| 

In [6]:
rfm2.describe("Monetary").show()

+-------+------------------+
|summary|          Monetary|
+-------+------------------+
|  count|           2064899|
|   mean|  996.391217129683|
| stddev|3678.6571150987147|
|    min|              0.31|
|    max| 790120.9400000002|
+-------+------------------+



In [9]:
sorted_users = rfm2.orderBy(desc('Monetary'))
sorted_users.show()

+---------+-------+---------+------------------+
|  user_id|Recency|Frequency|          Monetary|
+---------+-------+---------+------------------+
|553431815|      1|      888| 790120.9400000002|
|549109608|     25|     1383| 750149.9700000001|
|569333570|     18|      488| 689273.0899999996|
|513230794|     40|     1373| 647798.6500000001|
|595104624|     36|      458|         605792.74|
|513320236|      3|     1159| 567148.1000000001|
|610871788|      1|      407| 561013.4799999989|
|568782581|     53|     2120| 540447.6100000002|
|598224683|      1|     1059| 539601.6799999999|
|538216048|     51|      417|523403.12999999995|
|563599039|     78|      958|474648.45999999996|
|512386086|    134|      806|473119.38999999984|
|515428951|     52|      490|447804.55000000005|
|516010934|     35|      482|         403633.97|
|512409624|     38|      521| 386011.6199999999|
|518514099|     21|      410| 368759.5599999999|
|562104312|     19|      260| 351876.0299999998|
|514726585|     34| 

In [21]:
rfm2 = rfm2.na.drop()

In [22]:
rfm2.count()

2064899

In [24]:
def count_monetary_below_threshold(df, lower_bound, upper_bound):
    
    filtered_df = df.filter((col("Monetary") >= lower_bound) & (col("Monetary") <= upper_bound))
    
    return filtered_df.count()

In [10]:
average_monetary = rfm2.agg(avg('Monetary').alias('Average_Monetary'))


average_monetary.show()

+----------------+
|Average_Monetary|
+----------------+
|996.391217129683|
+----------------+



In [23]:
from pyspark.sql import SparkSession
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import ClusteringEvaluator




feature_columns = rfm2.columns[1:] 

# Assembler les colonnes de features en un seul vecteur de features
assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(rfm2)


In [30]:

# Séparer les données en 60% pour l'entraînement et 40% pour le test
training_data, test_data = data.randomSplit([0.6, 0.4], seed=1234)

# Entraîner le modèle K-means uniquement sur les données d'entraînement
kmeans = KMeans().setK(6).setSeed(1)  # Spécifier le nombre de clusters (k) et la graine (seed) pour la reproductibilité
model = kmeans.fit(training_data)

# Faire des prédictions sur les données de test
predictions = model.transform(test_data)

# Évaluer la performance du clustering en utilisant l'évaluateur de clustering
evaluator = ClusteringEvaluator()
silhouette = evaluator.evaluate(predictions)
print("Silhouette with squared euclidean distance = " + str(silhouette))

# Afficher les résultats du clustering
centers = model.clusterCenters()
print("Cluster Centers: ")
for center in centers:
    print(center)

# Arrêter la session Spark






Silhouette with squared euclidean distance = 0.9164002514982978
Cluster Centers: 
[ 91.32736519   2.20662947 463.80008394]
[3.87777778e+01 4.80583333e+02 3.32712071e+05]
[5.26140351e+01 1.63654971e+02 1.01027941e+05]
[  73.85195734   10.00541381 4008.15981987]
[   53.10647596    74.38895511 38145.15316647]
[   62.92846229    30.16307821 13710.96435963]


In [39]:
predictions.show()


+---------+-------+---------+------------------+--------------------+----------+
|  user_id|Recency|Frequency|          Monetary|            features|prediction|
+---------+-------+---------+------------------+--------------------+----------+
| 96369466|     12|        1|             50.71|    [12.0,1.0,50.71]|         0|
|128968633|     55|        4|            714.01|   [55.0,4.0,714.01]|         0|
|238417100|     80|        1|            116.32|   [80.0,1.0,116.32]|         0|
|286168756|     51|        1|             28.29|    [51.0,1.0,28.29]|         0|
|307478059|     92|        1|             33.46|    [92.0,1.0,33.46]|         0|
|310829160|     71|        1|            105.54|   [71.0,1.0,105.54]|         0|
|338689827|    130|        2|1110.3500000000001|[130.0,2.0,1110.3...|         0|
|355749922|     21|        1|            106.57|   [21.0,1.0,106.57]|         0|
|366237542|    155|        2|300.15999999999997|[155.0,2.0,300.15...|         0|
|369288911|     48|        1

In [ ]:
feature_columns = rfm2.columns[1:]  # Supposant que la première colonne est l'ID

assembler = VectorAssembler(inputCols=feature_columns, outputCol="features")
data = assembler.transform(rfm2)

# Conversion en Pandas DataFrame pour scikit-learn
pandas_df = data.select("features").toPandas()

# Standardisation des données (optionnel mais souvent recommandé pour DBSCAN)
scaler = StandardScaler()
pandas_df['scaled_features'] = scaler.fit_transform(pandas_df['features'].apply(lambda x: np.array(x.toArray())).tolist())

# Entraînement de DBSCAN avec scikit-learn
eps = 0.3  # Rayon de voisinage
min_samples = 5  # Nombre minimum de points pour former un cluster
dbscan = DBSCAN(eps=eps, min_samples=min_samples)
pandas_df['cluster'] = dbscan.fit_predict(pandas_df['scaled_features'])

# Évaluation du clustering (optionnel)
silhouette_score = metrics.silhouette_score(pandas_df['scaled_features'], pandas_df['cluster'])
print(f"Silhouette Score: {silhouette_score}")

# Convertir les prédictions en DataFrame Spark pour une utilisation ultérieure
predictions_spark = spark.createDataFrame(pd.DataFrame(pandas_df))

# Affichage des résultats du clustering dans Spark DataFrame
predictions_spark.show()


In [ ]:
rfm.write.parquet('/home/ghani/Bureau/Data/rfm.parquet')

In [ ]:
#combined_sample=spark.read.parquet('/home/ghani/Bureau/Data/combined_sample.parquet')


In [ ]:

#counts = combined_sample.select([count(col(c)).alias(c) for c in combined_sample.columns])
#counts.show()

In [ ]:

#category_code_distinc = combined_sample.select([countDistinct(col(c)).alias(c) for c in combined_sample.columns])

#category_code_distinc.show()



In [ ]:

#counts_null = combined_sample.select([sum(col(c).isNull().cast("int")).alias(c) for c in combined_sample.columns])
#counts_null.show()

In [ ]:

# counts_null_brand_category = combined_sample.select(
#     sum(when((col("category_code").isNull()) & (col("brand").isNull()), 1).otherwise(0)).alias("null_count")
# )
# counts_null_brand_category.show()

In [ ]:

#event_type_counts = combined_sample.groupBy("event_type").agg(count("*").alias("nombre"))
#event_type_counts=event_type_counts.toPandas()
#event_type_counts.head()



In [ ]:
#combined_sample.describe(['price']).show()


In [ ]:

#purchase_df=combined_sample.filter(col("event_type")=="purchase")
#purchase_df.describe(["price"]).show()

In [ ]:
# top_brands = combined_sample.groupBy("brand").count().orderBy(col("count").desc()).limit(10)
# top_brands_df = top_brands.toPandas()


# top_categories = combined_sample.groupBy("category_code").count().orderBy(col("count").desc()).limit(10)
# top_categories_df = top_categories.toPandas()


# plt.figure(figsize=(14, 7))

# plt.subplot(1, 2, 1)
# plt.pie(top_brands_df['count'], labels=top_brands_df['brand'], autopct='%1.1f%%', startangle=140)
# plt.title('Top 10 Brands')


# plt.subplot(1, 2, 2)
# plt.pie(top_categories_df['count'], labels=top_categories_df['category_code'], autopct='%1.1f%%', startangle=140)
# plt.title('Top 10 Categories')

# plt.tight_layout()
# plt.show()

In [ ]:

# total = event_type_counts['nombre'].sum()
# event_type_counts['percentage'] = (event_type_counts['nombre'] / total) * 100


# plt.figure(figsize=(10, 6))
# bars = plt.bar(event_type_counts['event_type'], event_type_counts['nombre'], color=['green', 'blue', 'red'])


# for bar, percentage in zip(bars, event_type_counts['percentage']):
#     yval = bar.get_height()
#     plt.text(bar.get_x() + bar.get_width()/2, yval, f'{percentage:.1f}%', va='bottom')  

# plt.xlabel('Event Type')
# plt.ylabel('Nombre')
# plt.title('Histogram of Event Types with Percentages')
# plt.show()

In [ ]:

# combined_sample = combined_sample.filter(~(col("category_code").isNull() & col("brand").isNull()))

# combined_sample.show()


In [ ]:
# counts_null_brand_category = combined_sample.select(
#     sum(when((col("category_code").isNull()) & (col("brand").isNull()), 1).otherwise(0)).alias("null_count")
# )
# counts_null_brand_category.show()

In [ ]:
# before=combined_sample.count()
# combined_sample = combined_sample.dropDuplicates()
# after=combined_sample.count()
# print(before-after)

In [ ]:
# combined_sample = combined_sample.filter((col("user_session").isNotNull()))


In [ ]:
# counts_null2 = combined_sample.select([sum(col(c).isNull().cast("int")).alias(c) for c in combined_sample.columns])
# counts_null2.show()

In [ ]:
# combined_sample.write.parquet('/home/ghani/Bureau/Data/combined_sample.parquet')

In [ ]:
# combined_sample = combined_sample.fillna({'category_code': 'unknown_category'})

In [ ]:
# pandas_df["cluster"].nunique()

In [ ]:
spark.stop()